In [1]:
# This notebook is to ease the experiementation process of having to constantly reload any NN models
# Import Libraries
import cv2 as cv
from time import time

import yaml
import sensors
import pandas as pd


In [2]:
def get_frame_filename(f, id):
    num_digit = len(str(f))
    frame_name = str(f)
    if num_digit == 1:
        frame_name = '000' + frame_name
    elif num_digit == 2:
        frame_name = '00' + frame_name
    elif num_digit == 3:
        frame_name = '0' + frame_name

    filename = "../data/Blender/cam_move/imgseq_human/" + frame_name + "_move" + str(id) + ".png"
    return filename

# User Settings
media = 'img_human' # ball or human

with open('config_detection.yml', 'r') as file:
    detection_config = yaml.safe_load(file)
detect_setup = detection_config['yolov5_setup1']

cam_move_dir = '../data/Blender/cam_move/'
with open(cam_move_dir + 'cam_move_info.yml', 'r') as file:
    cam_info = yaml.safe_load(file)

camA = sensors.cam(cam_info["cam_move" + str(1)])
camB = sensors.cam(cam_info["cam_move" + str(0)])

csv_fileA = cam_move_dir + camA.cam_output['poses']
csv_fileB = cam_move_dir + camB.cam_output['poses']

cam_posesA = pd.read_csv(csv_fileA, header = 0)
cam_posesB = pd.read_csv(csv_fileB, header = 0)


# Initialise OOI detection model
camA.detect_init(detect_setup)
camB.detect_init(detect_setup)

requirements: /home/luacghee20/YOLO/requirements.txt not found, check failed.


YOLOv5 🚀 v7.0-163-g016e046 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 7974MiB)

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 
YOLOv5 🚀 v7.0-163-g016e046 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 7974MiB)

Fusing layers... 


requirements: /home/luacghee20/YOLO/requirements.txt not found, check failed.


YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 


In [3]:
import importlib # For ipynb

import reid_known as reid
import reid_known_plots as reid_plt
import numpy as np
importlib.reload(reid)

""" Data Acquisition """
f = 290
frame_imgseqA = get_frame_filename(f, camA.id)
frame_imgseqB = get_frame_filename(f, camB.id)
camA.update_pose_csv(cam_posesA.loc[f])
camB.update_pose_csv(cam_posesB.loc[f])


# Images captured by cameras
imgA = cv.imread(frame_imgseqA)
imgB = cv.imread(frame_imgseqB)

""" Detection and Pre-processing """
results_ooiA = camA.detect_OOI(imgA) # OOIs Detection for camA


# permutation = [0, 10, 9, 8, 5, 4, 2, 3, 1, 7, 6]
# idx = np.empty_like(permutation)
# idx[permutation] = np.arange(len(permutation))
# results_ooiA[:] = results_ooiA[idx, :]
# original_id = np.array([range(len(results_ooiA))])
# results_ooiA[:, 0] = original_id



results_ooiB = camB.detect_OOI(imgB) # OOIs Detection for camB

""" Noise """
# Add Noise
# reid.add_gaussian_noise(camA)
# reid.add_gaussian_noise(camB) 

# Remove Noise
# reid.remove_gaussian_noise(camA)
# reid.remove_gaussian_noise(camB)


' Noise '

In [5]:
importlib.reload(reid)
importlib.reload(reid_plt)

import numpy as np

""" Association """
t2 = time() # Reid start time
results_ooiA_reid, results_ooiB_reid, transformed_OOIs = reid.local_reid(camA, camB, results_ooiA, results_ooiB)
t3 = time() # Reid end time
print('ReID Time: '+ str(t3 - t2))


""" Debug Plots """
reid_plt.plot_original_id(imgA, imgB, results_ooiA, results_ooiB)
reid_plt.plot_reid(imgA, imgB, results_ooiA_reid, results_ooiB_reid)
reid_plt.plot_results(camA, camB, results_ooiA_reid, results_ooiB_reid, transformed_OOIs)


# COLUMN    |     0       |       1      |       2      |     3    |     4     |      5     |   6   |     7     |     8     |    9    |   10    |   11    |  12  |  13  |
# MEANING   | ORIGINAL ID | BB_centroidX | BB_centroidY | BB_WIDTH | BB_HEIGHT | CONFIDENCE | CLASS | BB_floorX | BB_floorY | WORLD_X | WORLD_Y | WORLD_Z | SEEN | REID |



ReID Time: 0.0045659542083740234
